In [1]:
import numpy as np
import numba
from numba import prange

In [2]:
def arnoldi_iteration(A, b, n, verbose, matvec):
    h = np.zeros((n + 1, n))
    Q = np.zeros((n + 1, A.shape[0]))

    Q[0] = b / np.linalg.norm(b)
    for k in range(1, n + 1):
        v = matvec(A, Q[k - 1])
        for j in range(k):
            h[j, k - 1] = np.dot(Q[j], v)
            v -= h[j, k - 1] * Q[j]
        h[k, k - 1] = np.linalg.norm(v)
        if verbose:
            print(h[k, k - 1])
        Q[k] = v / h[k, k - 1]
    return h

In [3]:
def matvec_simple(A, b):
    n = b.size
    z = np.zeros(n)
    for i in range(n):
        for j in range(n):
            z[i] += A[i][j] * b[j]
    return z

@numba.jit(nopython=True)
def matvec_numba(A, b):
    n = b.size
    z = np.zeros(n)
    for i in range(n):
        for j in range(n):
            z[i] += A[i][j] * b[j]
    return z

@numba.jit(nopython=True, parallel=True)
def matvec_numba_parallel(A, b):
    n = b.size
    z = np.zeros(n)
    for i in prange(n):
        for j in range(n):
            z[i] += A[i][j] * b[j]
    return z

In [4]:
A_test = np.array([[2.0, 2.0], [2.0, 2.0]])
b_test = np.array([2.0, 2.0])
print(matvec_numba(A_test, b_test))
print(matvec_numba_parallel(A_test, b_test))

[8. 8.]
[8. 8.]


In [5]:
n = 5000
k = 4

In [6]:
np.random.seed(seed=1)
A = np.random.rand(n, n)
b = np.random.rand(n)

### Simple matvec

In [7]:
%%time
h = arnoldi_iteration(A, b, k, True, matvec_simple)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 1min 9s, sys: 0 ns, total: 1min 9s
Wall time: 1min 9s


### Simple numba matvec

In [8]:
%%time
h = arnoldi_iteration(A, b, k, True, matvec_numba)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 201 ms, sys: 194 µs, total: 201 ms
Wall time: 199 ms


### Paralleled numba matvec

In [9]:
numba.set_num_threads(4)

In [10]:
%%time
h = arnoldi_iteration(A, b, k, True, matvec_numba_parallel)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 308 ms, sys: 7.94 ms, total: 316 ms
Wall time: 84.4 ms


### Multiprocessing

In [11]:
from multiprocessing import Process, Array, Barrier, Queue
import ctypes

In [12]:
def worker(A, x, res, n,  l, r, task_queue, feedback_queue, stop):
    res = np.frombuffer(res, dtype=np.float64)
    x = np.frombuffer(x, dtype=np.float64)
    work = 1
    end = 2
    while True:
        task = task_queue.get() 
        if (task == end):
            break
        res[l : r] = 0.0
        for i in range(l, r):
            for j in range(n):
                res[i] += A[i][j] * x[j]
        feedback_queue.put(1)
        stop.wait() 

In [13]:
def arnoldi_iteration_multiprocessing(A, b, n, verbose, num_proc):
    h = np.zeros((n + 1, n))
    Q = np.zeros((n + 1, A.shape[0]))

    block_size = A.shape[0] // num_proc
    positions = [0 for i in range(num_proc + 1)]
    for i in range(num_proc):
        curr_amount = block_size
        if i < A.shape[0] % num_proc:
            curr_amount += 1
        positions[i + 1] = positions[i] + curr_amount

    work = 1
    end = 2

    task_queue = Queue()
    feedback_queue = Queue()
    stop = Barrier(num_proc)

    # array for res
    v_shared = Array(ctypes.c_double, A.shape[0], lock=False)
    v = np.frombuffer(v_shared, dtype=np.float64)
    
    # array for rhs
    Qj_shared = Array(ctypes.c_double, A.shape[0], lock=False)
    Qj = np.frombuffer(Qj_shared, dtype=np.float64)

    processes = []
    for i in range(num_proc):
        left_ind = positions[i]
        right_ind = positions[i + 1]
        processes.append(Process(target=worker, args=(A, Qj_shared, v_shared, A.shape[0], left_ind, right_ind, task_queue, feedback_queue, stop)))
        processes[i].start()

    def matvec(arr):
        np.copyto(Qj, arr)
        for i in range(num_proc):
            task_queue.put(work)
        for i in range(num_proc):
            feedback_queue.get()

    Q[0] = b / np.linalg.norm(b)

    for k in range(1, n + 1):
        matvec(Q[k - 1])
        for j in range(k):
            h[j, k - 1] = np.dot(Q[j], v)
            v -= h[j, k - 1] * Q[j]
        h[k, k - 1] = np.linalg.norm(v)
        if verbose:
            print(h[k, k - 1])
        Q[k] = v / h[k, k - 1]
    
    def end_processes():
        for i in range(num_proc):
            task_queue.put(end)
        for process in processes:
            process.join()
    
    end_processes()

    return h

In [14]:
%%time
h = arnoldi_iteration_multiprocessing(A, b, k, True, 4)

1083.1252052635512
23.67790056752565
20.47699083489922
20.650615698615393
CPU times: user 23.7 ms, sys: 40 ms, total: 63.6 ms
Wall time: 23.4 s
